<a href="https://colab.research.google.com/github/codeprogredire/face-mask-detection/blob/main/face_mask_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Import useful packages
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import cv2

In [3]:
!pip install gdown

In [4]:
!gdown --id 1cW6Nmt4Vl-DJ9yZyr7dE1FmYbyZAYAZx

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1cW6Nmt4Vl-DJ9yZyr7dE1FmYbyZAYAZx
From (redirected): https://drive.google.com/uc?id=1cW6Nmt4Vl-DJ9yZyr7dE1FmYbyZAYAZx&confirm=t&uuid=b63b7206-8259-445d-a8fc-5d38e2bfae89
To: /content/train.zip
100% 77.5M/77.5M [00:00<00:00, 106MB/s]


In [5]:
# The dataset is composed of images from WIDER Face (http://shuoyang1213.me/WIDERFACE/)
# and MAFA(http://www.escience.cn/people/geshiming/mafa.html)
# Once the images are downloaded from above two sites, they have been augmented to filtered out only faces, with mask and without mask

# Download train.zip from https://drive.google.com/file/d/1cW6Nmt4Vl-DJ9yZyr7dE1FmYbyZAYAZx/view?usp=sharing
# Then upload the zipped file from Left section and unzip images for training
!unzip -q /content/train.zip

In [6]:
!gdown --id 1Tyd9Ods8emWB2r6XHFAiRNu8dgVJuqWv

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1Tyd9Ods8emWB2r6XHFAiRNu8dgVJuqWv
From (redirected): https://drive.google.com/uc?id=1Tyd9Ods8emWB2r6XHFAiRNu8dgVJuqWv&confirm=t&uuid=b04c45ed-34c2-41c3-8799-be822c9e1f32
To: /content/validation.zip
100% 33.6M/33.6M [00:00<00:00, 57.2MB/s]


In [7]:
# The dataset is composed of images from WIDER Face (http://shuoyang1213.me/WIDERFACE/)
# and MAFA(http://www.escience.cn/people/geshiming/mafa.html)
# Once the images are downloaded from above two sites, they have been augmented to filtered out only faces, with mask and without mask


# Download validation.zip from https://drive.google.com/file/d/1Tyd9Ods8emWB2r6XHFAiRNu8dgVJuqWv/view?usp=sharing
# Then upload the zipped file from Left section and unzip images for validation
!unzip -q /content/validation.zip

In [8]:
PATH = "/content/"

In [9]:
# Defining directory path of training and validation
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [10]:
# Defining Two classes directory
train_face_dir = os.path.join(train_dir, 'face')
train_mask_dir = os.path.join(train_dir, 'mask')
validation_face_dir = os.path.join(validation_dir, 'face')
validation_mask_dir = os.path.join(validation_dir, 'mask')

In [11]:
# Calculating number of training and validation images
num_face_tr = len(os.listdir(train_face_dir))
num_mask_tr = len(os.listdir(train_mask_dir))

num_face_val = len(os.listdir(validation_face_dir))
num_mask_val = len(os.listdir(validation_mask_dir))

total_train = num_face_tr + num_mask_tr
total_val = num_face_val + num_mask_val

In [12]:
print(total_train)
print(total_val)

4430
1916


In [13]:
batch_size = 32
IMG_HEIGHT = 160
IMG_WIDTH = 160

In [14]:
#ImageDataGenerator accepts the original data, randomly transforms it, and returns only the new, transformed data
train_image_generator = ImageDataGenerator(rescale=1./255) # Normalize all pixel values between 0 & 1
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [15]:
# flow_from_directory Extract data from specified folder where your '2' or 'n' classes of folders are present
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

Found 4430 images belonging to 2 classes.
Found 1916 images belonging to 2 classes.


In [16]:
# Define Input shape of images for InceptionV3 model
IMG_SHAPE = (160, 160, 3)
base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

87910968/87910968 [==============================] - 0s 0us/step


In [17]:
# Printing total number of layers of InceptionV3 model
len(base_model.layers)

311

In [18]:
# base model architecture
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 160, 160, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 79, 79, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 79, 79, 32)           96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 79, 79, 32)           0         ['batch_normalizati

In [19]:
# Freeze the convolutional base before you compile and train the model
base_model.trainable = False

In [20]:
flatten_layer = tf.keras.layers.Flatten()
flatten_layer

In [21]:
# Flatten works by converting Matrix to single array/column
flatten_layer = tf.keras.layers.Flatten()
# Dense or fully-connected layers are used when association can exist among any feature to any other feature in data point
Dense1 = tf.keras.layers.Dense(units=500)  # default activation=None,i.e. Linear Matrix multiplication
# Batch normalization is used to apply normalization to the output of the hidden layers.
batch_normalization = tf.keras.layers.BatchNormalization()
# A dropout layer is used for regularization where you randomly set some of the dimensions of your input vector to be zero.
dropout = tf.keras.layers.Dropout(0.2)
# Output will be a Tensor representing the input tensor, transformed by the relu activation function
relu_activation1 = tf.keras.layers.ReLU()
Dense2 = tf.keras.layers.Dense(units=128)
batch_normalization2 = tf.keras.layers.BatchNormalization()
relu_activation2 = tf.keras.layers.ReLU() # standard ReLU activation: max(x, 0)
output = tf.keras.layers.Dense(units=1,activation='sigmoid') # i.e. output = activation(dot(input, kernel) + bias)

In [22]:
# # Now stack the feature extracted above layer by layer using a tf.keras.Sequential model
model = tf.keras.Sequential([
  base_model,
  flatten_layer,
  Dense1,
  batch_normalization,
    dropout,
relu_activation1,
  Dense2,
batch_normalization2,
relu_activation2,
    output
])

In [23]:
# Compile the model before training it. Since there are two classes, use a
# binary cross-entropy loss with from_logits=True since the model provides a linear output
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 3, 3, 2048)        21802784  
                                                                 
 flatten_1 (Flatten)         (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 500)               9216500   
                                                                 
 batch_normalization_94 (Ba  (None, 500)               2000      
 tchNormalization)                                               
                                                                 
 dropout (Dropout)           (None, 500)               0         
                                                                 
 re_lu (ReLU)                (None, 500)               0         
                                                        

In [25]:
# steps_per_epoch : it specifies the total number of steps taken from the generator
# epochs : an integer and number of epochs we want to train our model for.
# validation_data can be an inputs and targets list
# validation_steps:only if the validation_data is a generator then only this argument can be used
# fit_generator() function first accepts a batch of the dataset,
# then performs backpropagation on it, and then updates the weights in our model

model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train / batch_size,
    epochs=10,
    validation_data=val_data_gen,
    validation_steps=total_val / batch_size
)

<ipython-input-25-c203ce8a23f3>:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5818: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


138/138 [==============================] - 33s 144ms/step - loss: 0.1293 - accuracy: 0.9519 - val_loss: 0.3344 - val_accuracy: 0.9228
Epoch 2/10
138/138 [==============================] - 11s 78ms/step - loss: 0.0640 - accuracy: 0.9767 - val_loss: 0.4453 - val_accuracy: 0.9008
Epoch 3/10
138/138 [==============================] - 12s 87ms/step - loss: 0.0362 - accuracy: 0.9878 - val_loss: 0.3285 - val_accuracy: 0.9248
Epoch 4/10
138/138 [==============================] - 11s 77ms/step - loss: 0.0239 - accuracy: 0.9926 - val_loss: 0.4524 - val_accuracy: 0.9259
Epoch 5/10
138/138 [==============================] - 10s 74ms/step - loss: 0.0216 - accuracy: 0.9944 - val_loss: 0.6986 - val_accuracy: 0.8914
Epoch 6/10
138/138 [==============================] - 10s 74ms/step - loss: 0.0095 - accuracy: 0.9968 - val_loss: 0.5647 - val_accuracy: 0.9181
Epoch 7/10
138/138 [==============================] - 10s 74ms/step - loss: 0.0100 - accuracy: 0.9966 - val_loss: 0.6458 - val_accuracy: 0.9076
Ep

In [26]:
model.save("inception_face_mask_detection.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
# Testing our model for image classification
image = cv2.imread("/content/validation/mask/mask.test_00000009.jpeg")
image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
image = cv2.resize(image,(160,160))
img = tf.cast(image, tf.float32)
input_image = (img/255)
final_input_image = np.expand_dims(input_image, 0)
final_input_image.shape

(1, 160, 160, 3)

In [29]:
# Now load the trained model
loaded_model = tf.keras.models.load_model(filepath="inception_face_mask_detection.h5")

In [30]:
# Classify the image using prediction
result = loaded_model.predict(final_input_image)

1/1 [==============================] - 3s 3s/step


In [31]:
# Positive or negative random value will specify the class of image.
# In our case Positive values means Wearing mask and negative value means No mask
result

array([[1.]], dtype=float32)